In [ ]:
import math
import random
import numpy as np

# Try importing tqdm notebook
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError as e:
    print("tdqm.notebook not found. Try updating tdqm. Reverting to base tqdm", e)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import joblib  # To save models
from sklearn.metrics import hinge_loss
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

# CUSTOM IMPORTS
import LoaderFish  # function to gen custom point clouds (Wang et. al)

# utility functions
from src.utils import (
    chamfer_loss,
    HyperParameter,
    DirectorySetting,
)

# models
from src.models import AutoDecoder, CompNet, EnsembleCompNet

# datasets
from src.datasets import PointDriftDS, EncodingDS, PointNetDS

# train and test modules
from src.trainer import (
    find_encoding,
    train_decoder,
    eval_decoder,
    train_compnet,
    eval_compnet,
)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### Set Cuda device and SEED val

In [ ]:
device = None
use_cuda = torch.cuda.is_available()

if use_cuda:
    # inbuilt cudnn auto-tuner searches for best algorithm for hardware
    # cuddn.benchmark should be set to True when our input size does not vary
    torch.backends.cudnn.benchmark = True
    print("GPU training available")
    device = torch.device("cuda:0")
    print(f"Index of CUDA device in use is {torch.cuda.current_device()}")
else:
    print("GPU training NOT available")
    device = torch.device("cpu")
    print("Can only train on CPU")

DEBUG = True
SEED = 17 * 19

HP = HyperParameter(
    epochs=10,
    l2_reg=None,
    batch_size=16,
    num_point_cloud=3,
    encoding_iters=1000,
    encoding_size=256,
)
DS = DirectorySetting()

np.random.seed(SEED)
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

## Neural Network Modules

In [ ]:
class AutoDecoder(nn.Module):
    """
    AutoDecoder NN to learn point drift (latent encoding) between two 3D shapes
    """

    def __init__(self, encoding_dim=256, point_dim=3):
        super(AutoDecoder, self).__init__()
        self.fc1 = nn.Conv1d(encoding_dim + point_dim, 128, 1)
        self.fc2 = nn.Conv1d(128, 64, 1)
        self.fc3 = nn.Conv1d(64, point_dim, 1)

    def forward(self, X, encoding):
        num_points = X.shape[-1]  # num of points in each shape
        enc = encoding.unsqueeze(-1).repeat(1, 1, num_points)
        X_enc = torch.cat([X, enc], 1)
        X_enc = F.leaky_relu(self.fc1(X_enc))
        X_enc = F.leaky_relu(self.fc2(X_enc))

        # Return the drift from obj X determined by the latent encoding
        return X + self.fc3(X_enc)


class CompNet(nn.Module):
    """
    Ingests the latent encoding of two 3D objects
    and outputs the similarity score
    """

    def __init__(self, encoding_size=256):
        super(CompNet, self).__init__()
        self.fc1 = nn.Linear(encoding_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, encoding):
        X = F.leaky_relu(self.fc1(encoding))
        return torch.sigmoid(self.fc2(X))


class EnsembleCompNet(nn.Module):
    """
    Ingests the latent encoding of two 3D objects
    and outputs the similarity score using an ensemble of CompNets
    Stacked Ensemble
    """

    def __init__(
        self, comp_net=CompNet, num_ensemble=5, encoding_dim=256, seed_val=SEED
    ):
        """
        if comp_net is a module, EnsembleCompNet creates num_ensemble*comp_net NN modules
        if comp_net is a list of modules, EnsembleCompNet iterates through comp_net to get the NN modules
        """
        super(EnsembleCompNet, self).__init__()
        self.ensemble_compnet = nn.ModuleList()

        if isinstance(comp_net, list):
            if num_ensemble != len(comp_net):
                raise IndexError(
                    f"Length of comp_nets: {len(comp_net)} and num_ensemble: {num_ensemble} do not match"
                )
            comp_net_list = comp_net
            for i in range(num_ensemble):
                self.ensemble_compnet.append(comp_net_list[i])
        else:
            for i in range(num_ensemble):
                torch.manual_seed(seed_val * i + 1)
                if use_cuda:
                    torch.cuda.manual_seed(seed_val * i + 1)
                self.ensemble_compnet.append(comp_net(encoding_dim))
        self.final = nn.Linear(num_ensemble, 1)

    def forward(self, encoding):
        """Returns the final value of the results after a nn.Linear layer"""
        total_pred = torch.cat([net(encoding) for net in self.ensemble_compnet])
        total_pred = total_pred.reshape(-1, len(self.ensemble_compnet))

        return torch.sigmoid(self.final(total_pred))

## Dataset Modules and DataLoader implementations

In [ ]:
class PointNetDS(Dataset):
    """
    Create train dataset
    """

    def __init__(self, data, sampling_interval=3):
        # sample every sampling_interval-th point to speed up
        self.data = data.transpose((0, 2, 1))[:, :, ::sampling_interval]
        self.data = torch.from_numpy(self.data).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]


class PointDriftDS(Dataset):
    """
    Pairs each shape with one shape from the same class and one shape from a different class
    """

    def __init__(self, data, labels, sampling_interval=3):
        # sample every sampling_interval-th point to speed up
        self.data = data.transpose((0, 2, 1))[:, :, ::sampling_interval]
        self.labels = labels.squeeze()

        self.same_cls = []
        self.diff_cls = []
        idx_arr = np.arange(self.data.shape[0])
        same_idx = []
        diff_idx = []
        for i in range(self.labels.max() + 1):
            same_idx.append(idx_arr[self.labels == i])
            diff_idx.append(idx_arr[self.labels != i])
        for i in range(data.shape[0]):
            same = same_idx[self.labels[i]]
            diff = diff_idx[self.labels[i]]
            self.same_cls.append(same[random.randint(0, len(same) - 1)])
            self.diff_cls.append(diff[random.randint(0, len(diff) - 1)])
        self.data = torch.from_numpy(self.data).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        X = self.data[idx]
        same_cls_data = self.data[self.same_cls[idx]]
        diff_cls_data = self.data[self.diff_cls[idx]]

        return X, same_cls_data, diff_cls_data, idx


class EncodingDS(Dataset):
    """
    Generate encoding for each pair of shapes in the PointDriftDS
    """

    def __init__(self, PDDS, autodecoder, latent_size=256):
        self.PointDriftDS = PDDS
        self.autodecoder = autodecoder
        self.latent_size = latent_size
        self.same_cls = torch.zeros((len(self.PointDriftDS), latent_size))
        self.diff_cls = torch.zeros((len(self.PointDriftDS), latent_size))

    def train_encodings(self, num_iterations=50, lr=0.01, l2_reg=False, batch_size=16):
        dl = DataLoader(self.PointDriftDS, batch_size=batch_size, shuffle=False)
        i = 0
        batch_cnt = 0
        same_cls_loss = 0.0
        diff_cls_loss = 0.0
        self.autodecoder.eval()

        for batch_idx, (x, y, z, idx) in enumerate(dl):
            j = i + len(idx)
            loss, encoding = find_encoding(
                x,
                y,
                self.autodecoder,
                encoding_iters=num_iterations,
                encoding_size=self.latent_size,
                lr=lr,
                l2_reg=l2_reg,
            )
            same_cls_loss += loss
            self.same_cls[i:j] = encoding
            loss, encoding = find_encoding(
                x,
                z,
                self.autodecoder,
                encoding_iters=num_iterations,
                encoding_size=self.latent_size,
                lr=lr,
                l2_reg=l2_reg,
            )
            diff_cls_loss += loss
            self.diff_cls[i:j] = encoding

            i = j
            batch_cnt += 1
        print("Encodings trained")
        return (
            self.same_cls,
            self.diff_cls,
            same_cls_loss / batch_cnt,
            diff_cls_loss / batch_cnt,
        )

    def __len__(self):
        return len(self.PointDriftDS)

    def __getitem__(self, idx):
        return (*self.PointDriftDS[idx], self.same_cls[idx], self.diff_cls[idx])

## Module Train Functions

In [ ]:
def find_encoding(
    X, y, autodecoder, encoding_iters=300, encoding_size=256, lr=5e-4, l2_reg=False
):
    """
    Generate the encoding (latent vector) for each data in X
    """

    def _adjust_lr(initial_lr, optimizer, num_iters, decreased_by, adjust_lr_every):
        lr = initial_lr * ((1 / decreased_by) ** (num_iters // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

    decreased_by = 10
    adjust_lr_every = encoding_iters // 2

    encoding = (
        torch.ones(X.shape[0], encoding_size)
        .normal_(mean=0, std=1.0 / math.sqrt(encoding_size))
        .cuda()
    )

    encoding.requires_grad = True
    optimizer = torch.optim.Adam([encoding], lr=lr)
    loss_num = 0

    for i in range(encoding_iters):
        autodecoder.eval()
        _adjust_lr(lr, optimizer, i, decreased_by, adjust_lr_every)
        optimizer.zero_grad()
        y_pred = autodecoder(X, encoding)
        loss = chamfer_loss(y_pred, y, ps=y.shape[-1])

        if l2_reg:
            loss += 1e-4 * torch.mean(encoding.pow(2))
        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(i, loss.cpu().data.numpy(), encoding.norm())
        loss_num = loss.cpu().data.numpy()

    return loss_num, encoding

In [ ]:
def train_decoder(
    HP, DS, train_ds, test_ds=None, decoder=None, save_wt_fname="decoder.pth"
):
    """
    Default training is for 3D point dimensions

    Suggested Settings
        EPOCHS = 10
        point_dim = 3
        batch_size = 16
        learning_rate = 0.001
        encoding_size = 256

    Set save_wt_fname to None to disable weight saves
    """
    EPOCHS = HP.epochs
    point_dim = HP.num_point_cloud
    batch_size = HP.batch_size
    encoding_size = HP.encoding_size
    lr = HP.learning_rate

    if decoder is None:
        adnet = AutoDecoder(encoding_size, point_dim)
    else:
        adnet = decoder
    adnet = adnet.cuda()

    # encodings for same class transformation
    same_encoding = torch.nn.Embedding(len(train_ds), encoding_size, max_norm=1.0)
    # init encoding with Kaiming Initialization
    torch.nn.init.normal_(
        same_encoding.weight.data, 0.0, 1.0 / math.sqrt(encoding_size)
    )

    # encodings for different class transformation
    diff_encoding = torch.nn.Embedding(len(train_ds), encoding_size, max_norm=1.0)
    # init encoding with Kaiming Initialization
    torch.nn.init.normal_(
        diff_encoding.weight.data, 0.0, 1.0 / math.sqrt(encoding_size)
    )

    optimizer = torch.optim.Adam(
        [
            {
                "params": adnet.parameters(),
                "lr": lr,
            },
            {
                "params": same_encoding.parameters(),
                "lr": lr,
            },
            {
                "params": diff_encoding.parameters(),
                "lr": lr,
            },
        ]
    )

    op_schedule = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    adnet = nn.DataParallel(adnet)
    adnet.cuda()

    data_loader_train = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    for epoch in range(0, EPOCHS):
        adnet.train()
        same_total_loss = 0.0
        diff_total_loss = 0.0

        for batch_idx, (x, y, z, idx) in enumerate(data_loader_train):
            optimizer.zero_grad()
            x, y, z = x.cuda(), y.cuda(), z.cuda()
            x, y, z = (Variable(x).float(), Variable(y).float(), Variable(z).float())
            y_pred = adnet(x, same_encoding(torch.LongTensor(idx)))
            loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])
            same_total_loss += loss_cham.data.cpu().numpy()
            loss_cham.backward()

            z_pred = adnet(x, diff_encoding(torch.LongTensor(idx)))
            loss_cham = chamfer_loss(z, z_pred, ps=z.shape[-1])
            diff_total_loss += loss_cham.data.cpu().numpy()
            loss_cham.backward()

            optimizer.step()
            if batch_idx % 100 == 0 and batch_idx > 0:
                print(f"Epoch: {epoch}. batch_idx: {batch_idx}")
                print("Loss: ", same_total_loss / 100, diff_total_loss / 100)
                same_total_loss = 0.0
                diff_total_loss = 0.0
        op_schedule.step(epoch)

        if test_ds is not None and epoch % 5 == 0:
            print("Eval: ", eval_decoder(adnet, test_ds, batch_size=batch_size))

    if save_wt_fname is not None:
        torch.save(
            adnet.module.state_dict(),
            DS.AUTODECODER_TRAINED_WEIGHT_DIR + "/" + save_wt_fname,
        )
    return adnet

In [ ]:
def eval_decoder(decoder, eval_ds, batch_size=16):
    decoder.eval()
    encoding_ds = EncodingDS(eval_ds, decoder)
    return encoding_ds.train_encodings(
        num_iterations=10, lr=0.05, batch_size=batch_size
    )[2:]

In [ ]:
def train_svm_whole_dset(HP, DS, train_ds, save_wt_fname="svm_whole.pkl"):
    """
    Train the SVM

    Suggested Parameters
    batch_size=16
    """
    batch_size = HP.batch_size
    data_loader_train = DataLoader(train_ds, batch_size=16, shuffle=True)
    X, y = None, None
    # Combine the entire dataset
    for batch_idx, (_x, _y, _z, _idx, same_cls, diff_cls) in enumerate(
        data_loader_train
    ):
        same_cls, diff_cls = same_cls.detach().numpy(), diff_cls.detach().numpy()
        same_target, diff_target = (
            np.ones(same_cls.shape[0]),
            np.zeros(diff_cls.shape[0]),
        )

        if X is None and y is None:
            X = np.concatenate([same_cls, diff_cls], axis=0)
            y = np.concatenate([same_target, diff_target], axis=0)
        else:
            X = np.concatenate([X, same_cls, diff_cls], axis=0)
            y = np.concatenate([y, same_target, diff_target], axis=0)

    rbf_feature = RBFSampler(gamma=1, random_state=1)
    sgd_clf = SGDClassifier(loss="hinge", penalty="l2")

    X_features = rbf_feature.fit_transform(X)
    sgd_clf.fit(X_features, y)
    y_pred = sgd_clf.predict(X_features)

    print(f"Total Hinge Loss: {hinge_loss(y, y_pred)}")
    if save_wt_fname is not None:
        _ = joblib.dump(
            sgd_clf, DS.CLASSIFIER_TRAINED_WEIGHT_DIR + "/" + save_wt_fname, compress=9
        )
    return sgd_clf

In [ ]:
def eval_svm_whole_dset(svm_clf, test_ds, batch_size=16):
    print(len(test_ds))
    rbf_feature = RBFSampler(gamma=1, random_state=1)
    test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    X, y = None, None
    # Combine the entire dataset
    for batch_idx, (_x, _y, _z, _idx, same_cls, diff_cls) in enumerate(
        data_loader_train
    ):
        same_cls, diff_cls = same_cls.detach().numpy(), diff_cls.detach().numpy()
        same_target, diff_target = (
            np.ones(same_cls.shape[0]),
            np.zeros(diff_cls.shape[0]),
        )

        if X is None and y is None:
            X = np.concatenate([same_cls, diff_cls], axis=0)
            y = np.concatenate([same_target, diff_target], axis=0)
        else:
            X = np.concatenate([X, same_cls, diff_cls], axis=0)
            y = np.concatenate([y, same_target, diff_target], axis=0)

    X_features = rbf_feature.fit_transform(X)
    y_pred = svm_clf.predict(X_features)
    y, y_pred = y.astype(int), y_pred.astype(int)

    same_corr_cnt = np.sum(y_pred & y)
    same_incorr_cnt = np.sum(y & (y_pred ^ 1))
    diff_corr_cnt = np.sum((y ^ 1) & (y_pred ^ 1))
    diff_incorr_cnt = np.sum((y ^ 1) & y_pred)

    total_loss = hinge_loss(y, y_pred)
    if batch_idx % 100 == 0:
        print(f"Batch_idx: {batch_idx}")
        print("Cur loss: ", total_loss)

    precision = same_corr_cnt / (same_corr_cnt + diff_incorr_cnt)
    recall = same_corr_cnt / (same_corr_cnt + same_incorr_cnt)
    print("------------------ Evaluation Report ------------------")
    print(f"After {len(test_ds)} test points")
    print(f"Total Accuracy: {(same_corr_cnt + diff_corr_cnt) / (2 * len(test_ds))}")
    print()

    print("Metrics for the same class:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2 * precision * recall) / (precision + recall)}")

    precision = diff_corr_cnt / (diff_corr_cnt + same_incorr_cnt)
    recall = diff_corr_cnt / (diff_corr_cnt + diff_incorr_cnt)
    print()
    print("Metrics for the diff class:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2 * precision * recall) / (precision + recall)}")

    return (
        total_loss,
        same_corr_cnt,
        diff_corr_cnt,
        same_incorr_cnt,
        diff_incorr_cnt,
        len(test_ds),
    )

In [ ]:
def train_compnet(
    HP, DS, train_ds, test_ds=None, compnet=None, save_wt_fname="compnet.pth"
):
    """
    Train the CompNet

    Suggested Parameters
    EPOCHS=10
    batch_size=16
    encoding_size=256
    learning_rate=0.001
    """
    EPOCHS = HP.epochs
    point_dim = HP.num_point_cloud
    batch_size = HP.batch_size
    encoding_size = HP.encoding_size
    lr = HP.learning_rate

    if compnet is None:
        cpnet = CompNet(encoding_size=encoding_size)
    else:
        cpnet = compnet
    cpnet = cpnet.cuda()

    optimizer = torch.optim.Adam(cpnet.parameters(), lr=lr)
    op_schedule = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    cpnet = nn.DataParallel(cpnet)
    cpnet.cuda()

    data_loader_train = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    loss_fn = nn.BCELoss()
    for epoch in range(EPOCHS):
        same_total_loss = 0.0
        diff_total_loss = 0.0
        cpnet.train()
        for batch_idx, (x, y, z, idx, same_cls, diff_cls) in enumerate(
            data_loader_train
        ):
            optimizer.zero_grad()

            same_cls, diff_cls = same_cls.cuda(), diff_cls.cuda()
            same_cls, diff_cls = Variable(same_cls).float(), Variable(diff_cls).float()

            same_pred = cpnet(same_cls)
            same_target = torch.ones(same_pred.shape).float().cuda()
            same_loss = loss_fn(same_pred, same_target)
            same_loss.backward()
            same_total_loss += same_loss.data.cpu().numpy()

            diff_pred = cpnet(diff_cls)
            diff_target = torch.zeros(diff_pred.shape).float().cuda()
            diff_loss = loss_fn(diff_pred, diff_target)
            diff_loss.backward()
            diff_total_loss += diff_loss.data.cpu().numpy()

            optimizer.step()
            if batch_idx % 100 == 0 and batch_idx > 0:
                print(f"Epoch: {epoch}. batch_idx: {batch_idx}")
                print("Loss: ", same_total_loss / 100, diff_total_loss / 100)
                same_total_loss = 0.0
                diff_total_loss = 0.0
        op_schedule.step(epoch)

        if test_ds is not None and epoch % 5 == 0:
            print("Eval: ", eval_compnet(cpnet, test_ds, batch_size=batch_size))

    if save_wt_fname is not None:
        torch.save(
            cpnet.module.state_dict(),
            DS.CLASSIFIER_TRAINED_WEIGHT_DIR + "/" + save_wt_fname,
        )
    return cpnet

In [ ]:
def eval_compnet(cpnet, test_ds, batch_size=16, pred_threshold=0.5):
    cpnet.eval()
    test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    loss_fn = nn.BCELoss()

    same_total_loss = 0.0
    diff_total_loss = 0.0
    batch_cnt = 0
    same_corr_cnt = 0.0
    diff_corr_cnt = 0.0
    same_incorr_cnt = 0.0
    diff_incorr_cnt = 0.0

    for batch_idx, (x, y, z, idx, same_cls, diff_cls) in enumerate(test_dl):
        batch_cnt += 1
        same_cls, diff_cls = same_cls.cuda(), diff_cls.cuda()

        same_pred = cpnet(same_cls)
        same_target = torch.ones(same_pred.shape).float().cuda()
        same_loss = loss_fn(same_pred, same_target)
        same_total_loss += same_loss.data.cpu().numpy()

        same_corr_cnt += np.sum(same_pred.detach().cpu().numpy() > pred_threshold)
        same_incorr_cnt += np.sum(same_pred.detach().cpu().numpy() <= pred_threshold)

        diff_pred = cpnet(diff_cls)
        diff_target = torch.zeros(diff_pred.shape).float().cuda()
        diff_loss = loss_fn(diff_pred, diff_target)
        diff_total_loss += diff_loss.data.cpu().numpy()

        diff_corr_cnt += np.sum(diff_pred.detach().cpu().numpy() < pred_threshold)
        diff_incorr_cnt += np.sum(diff_pred.detach().cpu().numpy() >= pred_threshold)

    precision = same_corr_cnt / (same_corr_cnt + diff_incorr_cnt)
    recall = same_corr_cnt / (
        same_corr_cnt + same_incorr_cnt
    )  # same_corr_cnt / len(test_ds)
    print("------------------ Evaluation Report ------------------")
    print(f"Total Accuracy: {(same_corr_cnt + diff_corr_cnt) / (2 * len(test_ds))}")
    print(f"After {batch_cnt} batches and {len(test_ds)} test points")
    print()

    print("Metrics for the same class:")
    print(f"Avg loss: {same_total_loss / batch_cnt}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2 * precision * recall) / (precision + recall)}")

    precision = diff_corr_cnt / (diff_corr_cnt + same_incorr_cnt)
    recall = diff_corr_cnt / (
        diff_corr_cnt + diff_incorr_cnt
    )  # diff_corr_cnt / len(test_ds)
    print()
    print("Metrics for the diff class:")
    print(f"Avg loss: {diff_total_loss / batch_cnt}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {(2 * precision * recall) / (precision + recall)}")

    return (
        same_total_loss,
        diff_total_loss,
        same_corr_cnt,
        diff_corr_cnt,
        same_incorr_cnt,
        diff_incorr_cnt,
        batch_cnt,
        len(test_ds),
    )

_______

# Testing our AutoDecoder

## Generating dummy Train and Test pair Fish shapes using LoaderFish (Wang et al. 2019)

In [ ]:
CD_before = []  # chamfer distance before registration
CD_after = []  # chamfer distance after registration

train_size = 20000  # size for training data
test_size = 1000  # size for testing data
deformation_list = [0.5]
deformation = deformation_list[0]

print(".......Synthesizing Training Pairs......")
lf_train = LoaderFish.PointRegDataset(
    total_data=train_size,
    deform_level=deformation,
    noise_ratio=0,
    outlier_ratio=0,
    outlier_s=False,
    outlier_t=True,
    noise_s=False,
    noise_t=True,
    missing_points=0,
    miss_source=False,
    miss_targ=True,
)

data_loader_lf_train = torch.utils.data.DataLoader(
    lf_train, batch_size=16, shuffle=True
)

print(".......Synthesizing Testing Pairs......")
lf_test = LoaderFish.PointRegDataset(
    total_data=test_size,
    deform_level=deformation,
    noise_ratio=0,
    outlier_ratio=0,
    outlier_s=False,
    outlier_t=True,
    noise_s=False,
    noise_t=True,
    missing_points=0,
    miss_source=False,
    miss_targ=True,
)

data_loader_lf_test = torch.utils.data.DataLoader(lf_test, batch_size=10, shuffle=False)

In [ ]:
sample_data = next(iter(data_loader_lf_train))
print(
    len(sample_data), sample_data[0].shape, len(sample_data[0]), sample_data[0][0].shape
)

## Training the AutoDecoder on the sample generated LoaderFish dataset

We generate encoding latent vectors for the shapes generated by the LoaderFish

In [ ]:
# Hyperparameter Setup for loaderfish test
EPOCHS = 4
lr = 0.0001
encoding_size = 256
scheduler_gamma = 0.5
scheduler_step_sz = 30

In [ ]:
adnet = AutoDecoder(point_dim=2)
adnet.cuda()

encoding = torch.nn.Embedding(len(lf_train), encoding_size, max_norm=1.0)
# init encoding with Kaiming Initialization
torch.nn.init.normal_(encoding.weight.data, 0.0, 1.0 / math.sqrt(encoding_size))

optimizer = torch.optim.Adam(
    [
        {
            "params": adnet.parameters(),
            "lr": lr,
        },
        {
            "params": encoding.parameters(),
            "lr": lr,
        },
    ]
)
op_schedule = optim.lr_scheduler.StepLR(
    optimizer, step_size=scheduler_step_sz, gamma=scheduler_gamma
)

# use multiple gpus
adnet = nn.DataParallel(adnet)
adnet.cuda()

# Train the AutoDecoder adnet on the training data
for epoch in range(EPOCHS):
    adnet.train()
    total_loss = 0.0
    for batch_idx, (X, y, theta, _, idx) in enumerate(data_loader_lf_train):
        X, y, theta = X.cuda(), y.cuda(), theta.cuda()
        X, y, theta = Variable(X).float(), Variable(y).float(), Variable(theta).float()
        y_pred = adnet(X, encoding(torch.LongTensor(idx)))
        loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])

        optimizer.zero_grad()
        loss_cham.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch:{epoch} Batch Index:{batch_idx}")
            print(loss_cham.data.cpu().numpy())
    op_schedule.step(epoch)

In [ ]:
# Save weights of the decoder
torch.save(
    adnet.state_dict(), DS.AUTODECODER_TRAINED_WEIGHT_DIR + "/loaderfish_encoder01.pth"
)

## Testing the AudoDecoder network on the sample LoaderFish data

In [ ]:
adnet.eval()

for batch_idx, (X, y, theta, _, idx) in enumerate(data_loader_lf_test):
    X, y, theta = X.cuda(), y.cuda(), theta.cuda()
    X, y, theta = Variable(X).float(), Variable(y).float(), Variable(theta).float()
    loss_encoding, generated_encoding = find_encoding(
        X, y, adnet, encoding_iters=300, encoding_size=256
    )

    y_pred = adnet(X, generated_encoding)
    loss_cham = chamfer_loss(y, y_pred, ps=y.shape[-1])
    if batch_idx % 30 == 0:
        print(
            f"At batch:{batch_idx}, Chamfer Loss:{loss_cham}, Encoding Chamfer Loss:{loss_encoding}"
        )